In [1]:
import numpy as np
import pandas as pd
import math 
from pymoo.factory import get_problem
import GA
import statistics


pf = get_problem("truss2d").pareto_front()
phi1= pf[:,0].reshape(-1,1)
phi2= pf[:,1].reshape(-1,1)
x1=phi1/8.944
x2=2*x1
m =pf.shape[0]

In [2]:
def cnf(pop):
    pop=pd.DataFrame(pop)
    pop=pop.drop([1,3,5,7,8],axis=1)
    cnf=pop.to_numpy()
    return cnf

In [3]:
def niched_tournament_selection(pop,fitness):
    
    fitness=fitness.reshape((fitness.shape[0],1))
    pop_=np.array(np.concatenate((pop,fitness),axis=1))
    pop1_=np.array(pop_.copy())
    np.random.shuffle(pop1_)
    pop1=pop1_[:,:-1]
    parents=np.empty((pop.shape))
    pop2_=np.empty((pop1_.shape))
    a=cnf(pop)
    b=cnf(pop1)
    for i in range(pop_size):
        for j in range(pop_size):
            if (a[j]==b[i]).all():
                pop2_[i]=pop_[j]
        if pop1_[i,-1]>=pop2_[i,-1]:
            parents[i]=pop1_[i][:-1]
        else:
            parents[i]=pop2_[i][:-1]
            
    return parents

In [4]:
def fitness(pop):
    fitness_pop = np.empty([pop_size,1])
    fitness_pop[:]=np.nan
    
    d=pop[:,-1]
    d= d.astype(int)

    c= np.array((phi1**(pop[:,0]*pop[:,1]))*(phi2**(pop[:,2]*pop[:,3]))*(x1**(pop[:,4]*pop[:,5]))*(x2**(pop[:,6]*pop[:,7])))
    c.sort(axis=0)
    X=(c[-1,:]-c[0,:])/d
    
    for j in range(pop_size):
        C=0
        U=0
        cv=0
        flag=1
        minim=0
        temp=[]
        for i in range(d[j]):
            
            minim=c[:,j].min()
   
            points = c[:,j][(c[:,j]>=(minim+i*X[j]))*(c[:,j]<(minim+(i+1)*X[j]+0.00000000001))]
            count = points.size
            if count <= int(m/d[j]) : 
                U=U+count
                flag=1
                if len(temp)!=0:
                    cv+=statistics.stdev(temp)/statistics.mean(temp)
                    temp=[]
            else:
                temp.extend(points)
                if flag == 1:
                    C=C+1
                    flag=0
        
        b = [1,2,3,4]
        constr_violation = 0
        for i in range (len(b)):
            if 0.1 - abs(b[i])>0:
                constr_violation += 0.1 - abs(b[i])
        fitness_pop[j]=-(C+U+(100*cv))+1000*constr_violation   
    
    return fitness_pop

In [5]:
import random
pop_size=400
n_gen=1000

pop= np.empty([pop_size,9])
for i in range(pop_size):
    for j in range(8):
        if j%2==0:
            pop[i][j]=random.randint(0,1)
        else:
            if random.random()>=0.5:
                pop[i][j]=random.uniform(-1,-0.1)
            else :
                pop[i][j]=random.uniform(0.1,1)
                
    if (pop[i][0:7:2]==np.array([0,0,0,0])).all()==True:
        pop[i][0:7:2][random.randint(0,3)]=1

    pop[i][8]=random.randint(1,m)
    
    bmax=np.max(abs(pop[i][1:8:2]*pop[i][0:7:2]))
    pop[i][1:8:2]=pop[i][1:8:2]/bmax
    if np.min(pop[i][1:8:2]*pop[i][0:7:2])==-1:
        pop[i][1:8:2]=-pop[i][1:8:2]

        
    size=pop[:,1:8:2][abs(pop[:,1:8:2])>1].size
    if size!=0:
        t=np.append(np.random.uniform(-1,-0.1,int(size/2)),np.random.uniform(0.1,1,size-int(size/2)))
        np.random.shuffle(t)
        pop[:,1:8:2][abs(pop[:,1:8:2])>1]=t
        
f=fitness(pop)

In [6]:
for i in range(n_gen):
    
    parents=niched_tournament_selection(pop,f)  
    a=parents[:,0:7:2]
    b=parents[:,1:8:2]
    d=parents[:,-1]
    child=np.full((parents.shape),np.nan)
    #CROSSOVERS
    a=GA.sbx_int(a,1,10,0,1)
    b=GA.sbx_real(b,0.9,10)
    d=GA.sbx_int(d,1,10,1,1100)[:,0]
    
    #MUTATIONS
    child[:,0:7:2]=GA.mut_flip(a,0.15)
    child[:,1:8:2]=GA.mut_poly(b,50,-1,1,0.05)
    child[:,8]=GA.mut_int(d,50,1,1100,0.05)[:,0]
    
    for l in range(pop_size):
        
        if (child[l][0:7:2]==np.array([0,0,0,0])).all()==True:
            child[l][0:7:2][random.randint(0,3)]=1
        
        bmax=np.max(abs(child[l][1:8:2]*child[l][0:7:2]))
        child[l][1:8:2]=child[l][1:8:2]/bmax
        if np.min(child[l][1:8:2]*child[l][0:7:2])==-1:
            child[l][1:8:2]=-child[l][1:8:2]
        
    
            
    size=child[:,1:8:2][(abs(child[:,1:8:2])>1)+(abs(child[:,1:8:2])<0.1)].size
    if size!=0:
        t=np.append(np.random.uniform(-1,-0.1,int(size/2)),np.random.uniform(0.1,1,size-int(size/2)))
        np.random.shuffle(t)
        child[:,1:8:2][(abs(child[:,1:8:2])>1)+(abs(child[:,1:8:2])<0.1)]=t
        
    pop=child
    
    f=fitness(child)
    
    print("gen:",i+1,"=",child)

gen: 1 = [[ 1.00000000e+00  2.80930932e-01  0.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  6.95000000e+02]
 [ 1.00000000e+00 -5.36562728e-01  1.00000000e+00 ...  1.00000000e+00
  -6.32067886e-01  4.11000000e+02]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00 ...  0.00000000e+00
  -4.19528633e-01  8.17000000e+02]
 ...
 [ 1.00000000e+00  9.60373041e-01  0.00000000e+00 ...  0.00000000e+00
   2.17045558e-01  9.29000000e+02]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00 ...  0.00000000e+00
   4.74880701e-01  6.17000000e+02]
 [ 0.00000000e+00 -3.55991093e-01  0.00000000e+00 ...  0.00000000e+00
  -4.28932246e-01  4.20000000e+01]]
gen: 2 = [[ 0.00000000e+00 -8.08691308e-01  1.00000000e+00 ...  0.00000000e+00
   6.86960195e-01  3.97000000e+02]
 [ 1.00000000e+00 -3.13340767e-01  1.00000000e+00 ...  0.00000000e+00
   5.41737053e-01  6.13000000e+02]
 [ 0.00000000e+00  8.35268834e-01  1.00000000e+00 ...  0.00000000e+00
  -8.28919797e-01  6.10000000e+02]
 ...
 [ 0.00000000e+00  7.75509

In [7]:
#REDEFINING CLUSTERING CRITERION

pop=child
d=pop[:,-1]
d= d.astype(int)
unclustered_points=np.full((pop_size,1),np.nan)
clusters=np.full((pop_size,1),np.nan)
c =(phi1**(pop[:,0]*pop[:,1]))*(phi2**(pop[:,2]*pop[:,3]))*(x1**(pop[:,4]*pop[:,5]))*(x2**(pop[:,6]*pop[:,7]))                          
c.sort(axis=0)
X=(c[-1,:]-c[0,:])/d
for j in range(pop_size):
    C=0
    U=0
    cv=0
    flag=1
    minim=0
    temp=[]
    for i in range(d[j]):

        minim=c[:,j].min()

        points = c[:,j][(c[:,j]>=(minim+i*X[j]))*(c[:,j]<(minim+(i+1)*X[j]+0.00000000001))]
        count = points.size
        if count <= int(m/d[j])+3 : 
            U=U+count
            flag=1
            if len(temp)!=0:
                cv+=statistics.stdev(temp)/statistics.mean(temp)
                temp=[]
        else:
            temp.extend(points)
            if flag == 1:
                C=C+1
                flag=0
    

    unclustered_points[j]=U
    clusters[j]=C

In [33]:
a=pd.DataFrame((1100-unclustered_points)/11)
b=a[a>=70]
b.dropna(inplace=True)
b

,0
9,84.454545
18,73.000000
29,76.181818
74,92.545455
85,77.090909
91,79.181818
122,76.272727
128,100.000000
136,92.363636
138,71.636364


POINTS WITH STATISTICAL SIGNIFIACANCE>80% PRINTED BELOW

In [17]:
pop[9]

array([  0.        ,   0.84551368,   1.        ,   1.        ,
         0.        ,   0.97674822,   1.        ,   0.99311373,
       556.        ])

In [18]:
pop[74]

array([  1.        ,   0.99647626,   1.        ,   1.        ,
         0.        ,   0.94139721,   0.        ,  -0.71786868,
       552.        ])

In [19]:
pop[128]

array([ 0.00000000e+00,  7.50429377e-01,  0.00000000e+00,  4.37576385e-01,
        1.00000000e+00,  1.00000000e+00,  1.00000000e+00, -1.00000000e+00,
        5.52000000e+02])

In [20]:
pop[136]

array([  1.        ,   0.99776178,   1.        ,   1.        ,
         0.        ,   0.94588217,   0.        ,  -0.84012554,
       552.        ])

In [21]:
pop[303]

array([  1.        ,   0.99776178,   1.        ,   1.        ,
         0.        ,   0.94588217,   0.        ,  -0.84012554,
       552.        ])